# Analyse und Visualisierung eines Entscheidungsbaums für Predictive Maintenance
In diesem Notebook werden wir einen Entscheidungsbaum aus einem Random Forest-Modell für die vorausschauende Wartung (Predictive Maintenance) visualisieren und interpretieren. Wir verwenden einen Datensatz eines Industrieroboters, der verschiedene Sensorwerte und Betriebsdaten enthält.

## 1. Importieren der notwendigen Bibliotheken
Zunächst importieren wir die notwendigen Bibliotheken für Datenanalyse, Visualisierung und maschinelles Lernen.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import export_graphviz
import pydotplus
from IPython.display import Image
from google.colab import drive


## 2. Laden und Überprüfen der Daten
Wir laden den Datensatz und überprüfen die ersten Zeilen sowie einige grundlegende Statistiken und mögliche fehlende Werte.

In [10]:
# drive.mount('/content/drive')

file_path = 'https://raw.githubusercontent.com/eylift/AI-based_Predictive_Maintenance/main/industrieroboter_wartung_data.csv'
df = pd.read_csv(file_path)

# Anzeigen der ersten Zeilen der Daten
print(df.head())

# Grundlegende Statistiken
print(df.describe())

# Überprüfung auf fehlende Werte
print(df.isnull().sum())

   stromverbrauch_kwh  motorlast_prozent  temperatur_celsius  druck_bar  \
0           10.496714          78.577874           52.027862   5.378494   
1            9.861736          80.607845           57.003125   4.538917   
2           10.647689          85.830512           60.026218   5.434803   
3           11.523030          85.538021           60.234903   5.677819   
4            9.765847          61.223306           57.749673   5.206717   

   vibration_mm_s  betriebsstunden  wartung_notwendig  
0        3.281485      2699.677718                  1  
1        2.845187      2462.316841                  0  
2        3.028836      2029.815185                  1  
3        2.861317      1676.531611                  0  
4        2.869651      2349.111657                  0  
       stromverbrauch_kwh  motorlast_prozent  temperatur_celsius   druck_bar  \
count          200.000000         200.000000          200.000000  200.000000   
mean             9.959229          75.858682         

## 3. Datenvorbereitung
Wir definieren die Features und die Zielvariable und teilen die Daten in Trainings- und Testsets auf.

In [ ]:
X = df[['stromverbrauch_kwh', 'motorlast_prozent', 'temperatur_celsius', 'druck_bar', 'vibration_mm_s', 'betriebsstunden']]
y = df['wartung_notwendig']

# Aufteilen der Daten in Trainings- und Testset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Training des Random Forest Modells
Wir initialisieren und trainieren ein Random Forest Modell mit den Trainingsdaten.

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

## 5. Modellbewertung
Wir treffen Vorhersagen mit dem Testset und bewerten die Modellgenauigkeit, die Konfusionsmatrix und den Klassifikationsbericht.

In [ ]:
y_pred = model.predict(X_test)

# Berechnung der Genauigkeit
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# Konfusionsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', conf_matrix)

# Detaillierter Klassifikationsbericht
class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

## 6. Visualisierung der Konfusionsmatrix
Wir visualisieren die Konfusionsmatrix, um die Modellleistung besser zu verstehen.

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 7. Feature-Importances
Wir plotten die Wichtigkeit der einzelnen Features, um zu verstehen, welche Merkmale das Modell als am wichtigsten erachtet.

In [ ]:
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
feature_importances.sort_values().plot(kind='barh')
plt.title('Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.show()

## 8. Visualisierung eines Entscheidungsbaums
Wir wählen einen Entscheidungsbaum aus dem Random Forest und visualisieren ihn.

In [ ]:
estimator = model.estimators_[0]

# Exportieren des Entscheidungsbaums in das DOT-Format
dot_data = export_graphviz(estimator, out_file=None,
                           feature_names=X.columns,
                           class_names=['No Maintenance', 'Maintenance'],
                           filled=True, rounded=True,
                           special_characters=True)

# Erstellen eines Graphen aus dem DOT-Daten
graph = pydotplus.graph_from_dot_data(dot_data)

# Speichern des Entscheidungsbaums als PNG-Datei
graph.write_png('decision_tree.png')

# Anzeigen des gespeicherten Entscheidungsbaums
Image(filename='decision_tree.png')

## Interpretation des Entscheidungsbaums
Der Entscheidungsbaum ist ein grafisches Modell, das zeigt, wie Entscheidungen basierend auf den Eingabedaten getroffen werden. Jeder Knoten stellt eine Entscheidung basierend auf einem Merkmal und einem Schwellenwert dar, die zu zwei oder mehr Verzweigungen führt. Die Blätter am Ende der Zweige repräsentieren die endgültigen Entscheidungen oder Klassifizierungen.

- **Merkmal und Schwellenwert**: Jeder Knoten zeigt das Merkmal und den Schwellenwert, der verwendet wird, um die Daten zu teilen.
- **Gini-Index**: Ein Maß für die Unreinheit oder Unordnung des Knotens. Ein niedrigerer Wert bedeutet eine reinere Teilung.
- **Anzahl der Beispiele**: Die Anzahl der Datenpunkte, die zu diesem Knoten gehören.
- **Verteilung der Klassen**: Die Anzahl der Datenpunkte in den verschiedenen Klassen.
- **Vorhersage**: Die häufigste Klasse, die als Vorhersage verwendet wird.

Durch das Verständnis der Struktur und der Entscheidungen des Entscheidungsbaums können wir die Entscheidungsfindung des Modells besser interpretieren und nachvollziehen.